In [91]:
!docker-compose up -d

Creating network "clickhouse_default" with the default driver
Creating clickhouse ... 
ting clickhouse ... done

In [90]:
!docker-compose down

Stopping clickhouse ... 
ping clickhouse ... doneRemoving clickhouse ... 
Removing network clickhouse_default


In [92]:
from clickhouse_driver import Client

client = Client(host="localhost") 

In [93]:
client.execute("CREATE DATABASE IF NOT EXISTS benchmark")

[]

In [94]:
client.execute("DROP TABLE IF EXISTS benchmark.film_timestamp")

[]

In [95]:
create_sql = """
CREATE TABLE IF NOT EXISTS benchmark.film_timestamp (
      user_id UUID,
      film_id UUID,
      film_timestamp DateTime('Europe/Moscow'),
      event_time DateTime('Europe/Moscow'))
      Engine=MergeTree() PARTITION BY toYYYYMMDD(event_time) order by user_id;
"""
client.execute(create_sql)

[]

In [96]:
client.execute("select count(1) from benchmark.film_timestamp")

[(0,)]

In [97]:
import uuid
from faker import Faker


def generate_random_data(num_rows_per_chunk: int, iteration: int):
    chunk = []
    fake = Faker()

    event_time = fake.date_time_between()
    for i in range(iteration*num_rows_per_chunk):
        film_time = fake.date_time_between()
        
        chunk.append((str(uuid.uuid4()), str(uuid.uuid4()), film_time, event_time))

        if len(chunk) == num_rows_per_chunk:
            yield chunk
            chunk = []
            event_time = fake.date_time_between()

    yield chunk

In [98]:
import time
import statistics

class profile_code():
    def __init__(self):
        self.start = time.time()
        self.all_durations = []
        
    def setup_start_time(self):
        self.start = time.time()

    def checkpoint(self):
        end_time = time.time()
        self.all_durations.append(end_time - self.start)
        self.start = end_time

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        print(f"Median - {statistics.median(self.all_durations)}")
        avg = sum(self.all_durations) / len(self.all_durations)
        print(f"Average - {avg}")
        print(f"Summary - {sum(self.all_durations)}")

In [99]:
# Testing INSERT statments chunk with 500 records per time
with profile_code() as profiler:
    for chunk in generate_random_data(500, 2000):
        profiler.setup_start_time()
        client.execute(
                    f"INSERT INTO benchmark.film_timestamp  (user_id, film_id, film_timestamp, event_time) VALUES",
                    chunk,
                )
        profiler.checkpoint()

Median - 0.01758742332458496
Average - 0.017949770594286596
Summary - 35.91749095916748


In [100]:
# Testing INSERT statments online with 1 insert per time
with profile_code() as profiler:
    for chunk in generate_random_data(1, 2000):
        profiler.setup_start_time()
        client.execute(
                    f"INSERT INTO benchmark.film_timestamp  (user_id, film_id, film_timestamp, event_time) VALUES",
                    chunk,
                )
        profiler.checkpoint()

Median - 0.003395557403564453
Average - 0.003304220449322763
Summary - 6.611745119094849


In [101]:
num_uuids = 100
uuids = [str(u[0]) for u in client.execute(f"SELECT user_id FROM benchmark.film_timestamp LIMIT {num_uuids}")]

In [102]:
# Testing SELECT statments
with profile_code() as profiler:
    for u in uuids:
        profiler.setup_start_time()
        client.execute(f"SELECT * FROM benchmark.film_timestamp where user_id='{u}'")
        profiler.checkpoint()

Median - 0.03614664077758789
Average - 0.039163358211517334
Summary - 3.9163358211517334
